In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import DoubleType

In [2]:
spark = SparkSession.builder.getOrCreate()
df = spark.read.format("Json") \
  .option("inferSchema", "true") \
  .load("/FileStore/tables/TakeAway/data.json")
df.show()

+--------------------+--------------------+
 customerId| orders|
+--------------------+--------------------+
5b6950c008c899c1a...|[[[[10.24, 5b6950...|
5b6950c0624a4baa1...| []|
5b6950c0cf5f2f422...|[[[[14.46, 5b6950...|
5b6950c035a2fffb8...|[[[[11.48, 5b6950...|
5b6950c0e92fa253f...| []|
5b6950c0e4977d29c...|[[[[14.55, 5b6950...|
5b6950c0a9ddd50fc...|[[[[7.12, 5b6950c...|
5b6950c0128cd26db...|[[[[7.86, 5b6950c...|
5b6950c0ca6c53219...|[[[[9.41, 5b6950c...|
5b6950c025966a10c...|[[[[12.08, 5b6950...|
5b6950c0ae537a98c...|[[[[7.71, 5b6950c...|
5b6950c0f2f9a9c5e...|[[[[14.57, 5b6950...|
5b6950c097be63611...|[[[[13.69, 5b6950...|
5b6950c0544b331cd...|[[[[11.62, 5b6950...|
5b6950c056e426d04...|[[[[10.38, 5b6950...|
5b6950c0f32b7c4c5...|[[[[10.91, 5b6950...|
5b6950c0cd9150666...|[[[[14.67, 5b6950...|
5b6950c0234927877...|[[[[11.27, 5b6950...|
5b6950c0650b9b462...|[[[[6.05, 5b6950c...|
5b6950c0144ec028b...|[[[[11.06, 5b6950...|
+--------------------+--------------------+
only showing top 20 rows

In [3]:
df.printSchema()

root
-- customerId: string (nullable = true)
-- orders: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- basket: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- grossMerchandiseValueEur: double (nullable = true)
 | | | |-- productId: string (nullable = true)
 | | | |-- productType: string (nullable = true)
 | |-- orderId: string (nullable = true)

In [4]:
df1 = df.withColumn("orders", F.explode("orders")).select("customerId", F.col("orders.orderId").alias("orderId"), F.col("orders.basket").alias("basket")).drop("orders")
df1.show()

+--------------------+--------------------+--------------------+
 customerId| orderId| basket|
+--------------------+--------------------+--------------------+
5b6950c008c899c1a...|5b6950c0fda5be24b...|[[10.24, 5b6950c0...|
5b6950c008c899c1a...|5b6950c07ceef02d2...|[[7.73, 5b6950c0c...|
5b6950c008c899c1a...|5b6950c0b1589dd9d...|[[5.97, 5b6950c08...|
5b6950c008c899c1a...|5b6950c05bbe2e596...|[[5.97, 5b6950c0b...|
5b6950c008c899c1a...|5b6950c0340018c02...|[[11.34, 5b6950c0...|
5b6950c0cf5f2f422...|5b6950c087d2112e9...|[[14.46, 5b6950c0...|
5b6950c0cf5f2f422...|5b6950c06cb8daba7...|[[14.59, 5b6950c0...|
5b6950c035a2fffb8...|5b6950c0cbb9165f4...|[[11.48, 5b6950c0...|
5b6950c035a2fffb8...|5b6950c09136f9d2d...|[[6.61, 5b6950c0e...|
5b6950c035a2fffb8...|5b6950c0d5c98c311...|[[8.99, 5b6950c06...|
5b6950c035a2fffb8...|5b6950c0cbf764f1a...|[[10.65, 5b6950c0...|
5b6950c035a2fffb8...|5b6950c093830eb1b...|[[7.36, 5b6950c0a...|
5b6950c0e4977d29c...|5b6950c043b24d413...|[[14.55, 5b6950c0...|
5b6950c0e4977d29c...|5b6950c06538442f0...|[[10.08, 5b6950c0...|
5b6950c0e4977d29c...|5b6950c086fa7fd48...|[[7.53, 5b6950c03...|
5b6950c0a9ddd50fc...|5b6950c06c7a9c592...|[[7.12, 5b6950c08...|
5b6950c0a9ddd50fc...|5b6950c001254075c...|[[10.61, 5b6950c0...|
5b6950c0a9ddd50fc...|5b6950c0f9c6197b7...|[[10.47, 5b6950c0...|
5b6950c0a9ddd50fc...|5b6950c06308619b9...|[[8.64, 5b6950c0a...|
5b6950c0128cd26db...|5b6950c0ae35cb008...|[[7.86, 5b6950c0a...|
+--------------------+--------------------+--------------------+
only showing top 20 rows

In [5]:
df2 = df1.withColumn("basket", F.explode("basket").alias("basket")).select("customerId", "orderId", F.col("basket.productId").alias("productId"), F.col("basket.productType").alias("productType"), F.col("basket.grossMerchandiseValueEur").alias("grossMerchandiseValueEur"))
df2.show(truncate = False)

+------------------------+------------------------+------------------------+-----------+------------------------+
customerId |orderId |productId |productType|grossMerchandiseValueEur|
+------------------------+------------------------+------------------------+-----------+------------------------+
5b6950c008c899c1a4caf2a1|5b6950c0fda5be24b51cfe47|5b6950c02b3b80a9472b36ea|hot food |10.24 |
5b6950c008c899c1a4caf2a1|5b6950c0fda5be24b51cfe47|5b6950c0116450974d3ea65d|hot food |5.41 |
5b6950c008c899c1a4caf2a1|5b6950c0fda5be24b51cfe47|5b6950c03d34d984f478c88f|cold food |10.21 |
5b6950c008c899c1a4caf2a1|5b6950c07ceef02d2044eb8d|5b6950c0cafabfe0a98a6fe0|cold food |7.73 |
5b6950c008c899c1a4caf2a1|5b6950c07ceef02d2044eb8d|5b6950c087221cc876f0de59|cold food |11.85 |
5b6950c008c899c1a4caf2a1|5b6950c07ceef02d2044eb8d|5b6950c01601eb59ecc843bb|cold food |11.57 |
5b6950c008c899c1a4caf2a1|5b6950c0b1589dd9d3c9758c|5b6950c08f53e18f88ff70b8|hot food |5.97 |
5b6950c008c899c1a4caf2a1|5b6950c05bbe2e59661c9eae|5b6950c0b8b671f86c9404dc|beverage |5.97 |
5b6950c008c899c1a4caf2a1|5b6950c05bbe2e59661c9eae|5b6950c032678a1d97ef195d|hot food |10.28 |
5b6950c008c899c1a4caf2a1|5b6950c0340018c025fa6acf|5b6950c0c1d4b5ec14815cd1|hot food |11.34 |
5b6950c008c899c1a4caf2a1|5b6950c0340018c025fa6acf|5b6950c082bb19e3630ef55f|cold food |8.6 |
5b6950c0cf5f2f422a7d3a62|5b6950c087d2112e99fcf50a|5b6950c03b12b7974a9544b0|cold food |14.46 |
5b6950c0cf5f2f422a7d3a62|5b6950c087d2112e99fcf50a|5b6950c0d0193eb1ffff08ab|hot food |14.66 |
5b6950c0cf5f2f422a7d3a62|5b6950c06cb8daba7b3ce483|5b6950c0fd31bc635558e923|cold food |14.59 |
5b6950c0cf5f2f422a7d3a62|5b6950c06cb8daba7b3ce483|5b6950c034ed4eab311d27f1|cold food |9.0 |
5b6950c035a2fffb8d1b3c67|5b6950c0cbb9165f4cc2b575|5b6950c005e0a74b6021104c|beverage |11.48 |
5b6950c035a2fffb8d1b3c67|5b6950c0cbb9165f4cc2b575|5b6950c0b21e24a21734b000|beverage |5.91 |
5b6950c035a2fffb8d1b3c67|5b6950c09136f9d2d923640a|5b6950c0ef33bdb995b3f562|cold food |6.61 |
5b6950c035a2fffb8d1b3c67|5b6950c09136f9d2d923640a|5b6950c0743bfdb94f2e946f|cold food |6.27 |
5b6950c035a2fffb8d1b3c67|5b6950c0d5c98c311537fe48|5b6950c06a5f18f95ecc6538|beverage |8.99 |
+------------------------+------------------------+------------------------+-----------+------------------------+
only showing top 20 rows

In [6]:
def netMerchCal(x, y):
  if x == "beverage":
    return y+0.09*y
  elif x == "hot food":
    return y+0.15*y
  else:
    return y+0.07*y
  
netMerch_udf = F.udf(lambda a, b : netMerchCal(a, b), DoubleType())

df3 = df2.withColumn("netMerch", netMerch_udf("productType", "grossMerchandiseValueEur"))
df3.show(truncate = False)

+------------------------+------------------------+------------------------+-----------+------------------------+------------------+
customerId |orderId |productId |productType|grossMerchandiseValueEur|netMerch |
+------------------------+------------------------+------------------------+-----------+------------------------+------------------+
5b6950c008c899c1a4caf2a1|5b6950c0fda5be24b51cfe47|5b6950c02b3b80a9472b36ea|hot food |10.24 |11.776 |
5b6950c008c899c1a4caf2a1|5b6950c0fda5be24b51cfe47|5b6950c0116450974d3ea65d|hot food |5.41 |6.2215 |
5b6950c008c899c1a4caf2a1|5b6950c0fda5be24b51cfe47|5b6950c03d34d984f478c88f|cold food |10.21 |10.924700000000001|
5b6950c008c899c1a4caf2a1|5b6950c07ceef02d2044eb8d|5b6950c0cafabfe0a98a6fe0|cold food |7.73 |8.2711 |
5b6950c008c899c1a4caf2a1|5b6950c07ceef02d2044eb8d|5b6950c087221cc876f0de59|cold food |11.85 |12.679499999999999|
5b6950c008c899c1a4caf2a1|5b6950c07ceef02d2044eb8d|5b6950c01601eb59ecc843bb|cold food |11.57 |12.379900000000001|
5b6950c008c899c1a4caf2a1|5b6950c0b1589dd9d3c9758c|5b6950c08f53e18f88ff70b8|hot food |5.97 |6.8655 |
5b6950c008c899c1a4caf2a1|5b6950c05bbe2e59661c9eae|5b6950c0b8b671f86c9404dc|beverage |5.97 |6.5073 |
5b6950c008c899c1a4caf2a1|5b6950c05bbe2e59661c9eae|5b6950c032678a1d97ef195d|hot food |10.28 |11.822 |
5b6950c008c899c1a4caf2a1|5b6950c0340018c025fa6acf|5b6950c0c1d4b5ec14815cd1|hot food |11.34 |13.041 |
5b6950c008c899c1a4caf2a1|5b6950c0340018c025fa6acf|5b6950c082bb19e3630ef55f|cold food |8.6 |9.202 |
5b6950c0cf5f2f422a7d3a62|5b6950c087d2112e99fcf50a|5b6950c03b12b7974a9544b0|cold food |14.46 |15.4722 |
5b6950c0cf5f2f422a7d3a62|5b6950c087d2112e99fcf50a|5b6950c0d0193eb1ffff08ab|hot food |14.66 |16.859 |
5b6950c0cf5f2f422a7d3a62|5b6950c06cb8daba7b3ce483|5b6950c0fd31bc635558e923|cold food |14.59 |15.6113 |
5b6950c0cf5f2f422a7d3a62|5b6950c06cb8daba7b3ce483|5b6950c034ed4eab311d27f1|cold food |9.0 |9.63 |
5b6950c035a2fffb8d1b3c67|5b6950c0cbb9165f4cc2b575|5b6950c005e0a74b6021104c|beverage |11.48 |12.513200000000001|
5b6950c035a2fffb8d1b3c67|5b6950c0cbb9165f4cc2b575|5b6950c0b21e24a21734b000|beverage |5.91 |6.4419 |
5b6950c035a2fffb8d1b3c67|5b6950c09136f9d2d923640a|5b6950c0ef33bdb995b3f562|cold food |6.61 |7.0727 |
5b6950c035a2fffb8d1b3c67|5b6950c09136f9d2d923640a|5b6950c0743bfdb94f2e946f|cold food |6.27 |6.7089 |
5b6950c035a2fffb8d1b3c67|5b6950c0d5c98c311537fe48|5b6950c06a5f18f95ecc6538|beverage |8.99 |9.799100000000001 |
+------------------------+------------------------+------------------------+-----------+------------------------+------------------+
only showing top 20 rows

In [7]:
df3.groupBy("orderId").agg(F.sum("netMerch")).show(truncate = False)

+------------------------+------------------+
orderId |sum(netMerch) |
+------------------------+------------------+
5b6950c00e2a52bd0942f06c|37.379 |
5b6950c0162f624d9010b322|37.1877 |
5b6950c08198d994755a8216|33.9973 |
5b6950c0c799adc99ce9c40d|21.6729 |
5b6950c002178e70fe75023c|32.724900000000005|
5b6950c0f9c6197b78a0ee1c|12.040500000000002|
5b6950c0ab7c6d0dde5be392|15.582500000000001|
5b6950c0c300e1130ecaef18|14.2136 |
5b6950c05fa64323e7788b8b|16.1948 |
5b6950c044c4de796479e17f|42.075 |
5b6950c0f41211d950e8d2e2|32.6854 |
5b6950c0e864665202078b11|15.729 |
5b6950c042ce5e348c26185c|14.651 |
5b6950c0c359f625fa32723c|24.503 |
5b6950c035384b15cfa133d3|10.626 |
5b6950c075ae37c1b969c5aa|28.7952 |
5b6950c091c2bd7c6340ecaa|9.4176 |
5b6950c03da5792437f9dce7|6.9805 |
5b6950c07754fab9b24a0a8a|19.2945 |
5b6950c06f0ca78be02e9adf|13.9956 |
+------------------------+------------------+
only showing top 20 rows

In [8]:
df3.write.format("csv").save("/FileStore/tables/TakeAway/result/")